# Working With RasterLayers

The purpose of `RasterLayer` is to store and format data to produce a `TiledRasterLayer`. Thus, this class lacks the methods needed to perform any kind of spatial analysis. It can be thought of as something of an “organizer”. Which sorts and transfroms the data so that perform operations can be performed.

This following guide will go over the creation methods and abilities unique to `RasterLayer`. For `TiledRasterLayer`, please see this [guide] and for a more general overview of layers, look [here].

## Setting up the Environment for This Guide

In [ ]:
!curl -o /tmp/cropped.tif https://s3.amazonaws.com/geopyspark-test/example-files/cropped.tif

In [ ]:
import numpy as np
import geopyspark as gps

from pyspark import SparkContext

In [ ]:
conf = gps.geopyspark_conf(master="local[*]", appName="raster-layer-examples")
pysc = SparkContext(conf=conf)

## Creating RasterLayers

There are just two ways to create a `RasterLayer`: (1) through reading GeoTiffs from the local file system, S3, or HDFS; or (2) from an existing PySpark RDD.

### From GeoTiffs

The `get` method in the `geopyspark.geotrellis.geotiff` module creates an instance of `RasterLayer` from GeoTiffs. These files can be located on either your local file system, HDFS, or S3. In this example, a GeoTiff with spatial data is read locally.

In [ ]:
raster_layer = gps.geotiff.get(layer_type=gps.LayerType.SPATIAL, uri="file:///tmp/cropped.tif")
raster_layer

**Note**: If you have multiple GeoTiffs, you can just specify the directory where they’re all stored. Or if the GeoTiffs are spread out in multiple locations, you can give get a list of the places to read in the GeoTiffs.

### From PySpark RDDs

The second option is to create a `RasterLayer` from a PySpark `RDD` via the `from_numpy_rdd` class method. This step is a bit more involved than the last, as it requires the data within the PySpark RDD to be formatted in a specific way (see [How is Data Stored and Represented in GeoPySpark](core-concepts.ipynb#How-is-Data-Stored-and-Represented-in-GeoPySpark?) for more information).

The following example constructs a RDD with a tuple. The first element is a `ProjectedExtent` because we have decidec to make the data spatial. If we were dealing with spatial-temproal data, then `TemporalProjectedExtent` would be the first element. `Tile` will always be the second element of the tuple.

In [ ]:
arr = np.ones((1, 16, 16), dtype=int)
tile = gps.Tile.from_numpy_array(numpy_array=np.array(arr), no_data_value=-500)

extent = gps.Extent(0.0, 1.0, 2.0, 3.0)
projected_extent = gps.ProjectedExtent(extent=extent, epsg=3857)

rdd = pysc.parallelize([(projected_extent, tile)])
gps.RasterLayer.from_numpy_rdd(layer_type=gps.LayerType.SPATIAL, numpy_rdd=rdd)

## Using RasterLayers

Once we've initialized our `RasterLayer` instance, it is now time to use it.

## Collecting Metadata

In [ ]:
# Collecting Metadata with the default LocalLayout()
metadata = raster_layer.collect_metadata()
metadata

In [ ]:
# Collecting Metadata with the default GlobalLayout()
raster_layer.collect_metadata(layout=gps.GlobalLayout())

In [ ]:
# Collecting Metadata with a LayoutDefinition
extent = gps.Extent(0.0, 0.0, 33.0, 33.0)
tile_layout = gps.TileLayout(2, 2, 256, 256)
layout_definition = gps.LayoutDefinition(extent, tile_layout)

raster_layer.collect_metadata(layout=layout_definition)

## Tiling Data to a Layout

`tile_to_layout` will tile and format the rasters within a ``RasterLayer`` to a given layout. The layout to tile to can be derived from various sources.

### From Metadata

In [ ]:
tiled_raster_layer = raster_layer.tile_to_layout(layout=metadata)
tiled_raster_layer

### From LayoutDefinition

In [ ]:
raster_layer.tile_to_layout(layout=layout_definition)

### From A TiledRasterLayer

One can tile a `RasterLayer` to the same layout as a `TiledRasterLayout`.

In [ ]:
raster_layer.tile_to_layout(layout=tiled_raster_layer)

### From LocalLayout

In [ ]:
raster_layer.tile_to_layout(gps.LocalLayout())

### From GlobalLayout

In [ ]:
raster_layer.tile_to_layout(gps.GlobalLayout())

### Resampling During Tiling

It is also possible to select a `resample_method` when tiling the layer.

In [ ]:
raster_layer.tile_to_layout(layout=metadata, resample_method=gps.ResampleMethod.BILINEAR)